In [1]:
from pyspark import HiveContext
from pyspark import SparkContext
from pyspark.sql import SparkSession

In [2]:
sc = SparkContext()

21/11/11 13:38:57 WARN Utils: Your hostname, shivam-Vostro-3559 resolves to a loopback address: 127.0.1.1; using 192.168.0.105 instead (on interface wlx7cc2c61989aa)
21/11/11 13:38:57 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
21/11/11 13:39:00 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
hc = HiveContext(sc)

In [6]:
conf_out = sc.getConf()
print(conf_out.toDebugString())

spark.app.id=local-1636618147332
spark.app.name=pyspark-shell
spark.app.startTime=1636618142387
spark.driver.host=192.168.0.105
spark.driver.port=44475
spark.executor.id=driver
spark.master=local[*]
spark.rdd.compress=True
spark.serializer.objectStreamReset=100
spark.sql.catalogImplementation=hive
spark.submit.deployMode=client
spark.submit.pyFiles=
spark.ui.showConsoleProgress=true


In [4]:
hc.sql("CREATE DATABASE IF NOT EXISTS OFFICE")

21/11/07 11:39:43 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
21/11/07 11:39:43 WARN HiveConf: HiveConf of name hive.stats.retries.wait does not exist
21/11/07 11:39:52 WARN ObjectStore: Version information not found in metastore. hive.metastore.schema.verification is not enabled so recording the schema version 2.3.0
21/11/07 11:39:52 WARN ObjectStore: setMetaStoreSchemaVersion called but recording version is disabled: version = 2.3.0, comment = Set by MetaStore shivam@127.0.1.1
21/11/07 11:39:53 WARN ObjectStore: Failed to get database global_temp, returning NoSuchObjectException


DataFrame[]

In [5]:
hc.sql("drop table if exists office.employee")

DataFrame[]

In [6]:
hc.sql("CREATE TABLE IF NOT EXISTS OFFICE.EMPLOYEE(id INT, name STRING, age STRING) ROW FORMAT DELIMITED FIELDS TERMINATED BY ',' LINES TERMINATED BY '\n'")

21/11/07 11:39:56 WARN SessionState: METASTORE_FILTER_HOOK will be ignored, since hive.security.authorization.manager is set to instance of HiveAuthorizerFactory.
21/11/07 11:39:56 WARN HiveConf: HiveConf of name hive.internal.ss.authz.settings.applied.marker does not exist
21/11/07 11:39:56 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
21/11/07 11:39:56 WARN HiveConf: HiveConf of name hive.stats.retries.wait does not exist
21/11/07 11:39:56 WARN HiveMetaStore: Location: file:/home/shivam/notebook/spark-warehouse/office.db/employee specified for non-external table:employee


DataFrame[]

In [7]:
hc.sql("LOAD DATA LOCAL INPATH 'data/employee.txt' INTO TABLE OFFICE.EMPLOYEE")

DataFrame[]

In [8]:
hc.sql("""select * from OFFICE.EMPLOYEE""").show()

+----+--------+---+
|  id|    name|age|
+----+--------+---+
|1201|  satish| 25|
|1202| krishna| 28|
|1203|   amith| 39|
|1204|   javed| 23|
|1205|  prudvi| 33|
|1206|  drudvi| 26|
|1202|   krudi| 20|
|1203|  drudvi| 27|
|1202|  nrudvi| 29|
|1201|  mrudvi| 35|
+----+--------+---+



In [9]:
hc.sql("""create view if not exists office.v_emp as 
        select id, avg(age) as average_age, count(id) as num_persons
        from office.employee 
        group by id
        order by id desc""");
hc.sql("""select * from office.v_emp""").show()

+----+------------------+-----------+
|  id|       average_age|num_persons|
+----+------------------+-----------+
|1206|              26.0|          1|
|1205|              33.0|          1|
|1204|              23.0|          1|
|1203|              33.0|          2|
|1202|25.666666666666668|          3|
|1201|              30.0|          2|
+----+------------------+-----------+



In [10]:
hc.sql("""describe office.employee""").show()

+--------+---------+-------+
|col_name|data_type|comment|
+--------+---------+-------+
|      id|      int|   null|
|    name|   string|   null|
|     age|   string|   null|
+--------+---------+-------+



In [11]:
hc.sql("""describe formatted office.employee""").show(100,truncate=False)

+----------------------------+-------------------------------------------------------------+-------+
|col_name                    |data_type                                                    |comment|
+----------------------------+-------------------------------------------------------------+-------+
|id                          |int                                                          |null   |
|name                        |string                                                       |null   |
|age                         |string                                                       |null   |
|                            |                                                             |       |
|# Detailed Table Information|                                                             |       |
|Database                    |office                                                       |       |
|Table                       |employee                                                     

In [15]:
hc.sql("""create table IF NOT EXISTS office.emp_copy as select * from office.employee""")

21/11/07 12:52:55 WARN ResolveSessionCatalog: A Hive serde table will be created as there is no table provider specified. You can set spark.sql.legacy.createHiveTableByDefault to false so that native data source table will be created instead.
21/11/07 12:52:55 WARN HiveMetaStore: Location: file:/home/shivam/notebook/spark-warehouse/office.db/emp_copy specified for non-external table:emp_copy


DataFrame[]

In [16]:
hc.sql("""select * from office.emp_copy""").show()

+----+--------+---+
|  id|    name|age|
+----+--------+---+
|1201|  satish| 25|
|1202| krishna| 28|
|1203|   amith| 39|
|1204|   javed| 23|
|1205|  prudvi| 33|
|1206|  drudvi| 26|
|1202|   krudi| 20|
|1203|  drudvi| 27|
|1202|  nrudvi| 29|
|1201|  mrudvi| 35|
+----+--------+---+



In [17]:
hc.sql("""describe formatted office.emp_copy""").show(100,False)

+----------------------------+-------------------------------------------------------------+-------+
|col_name                    |data_type                                                    |comment|
+----------------------------+-------------------------------------------------------------+-------+
|id                          |int                                                          |null   |
|name                        |string                                                       |null   |
|age                         |string                                                       |null   |
|                            |                                                             |       |
|# Detailed Table Information|                                                             |       |
|Database                    |office                                                       |       |
|Table                       |emp_copy                                                     

In [18]:
hc.sql("""insert into table office.emp_copy 
        select * from office.employee""").show()

## this will append the contents of employee table to emp_copy table

++
||
++
++



In [19]:
hc.sql("""select count(*) from office.emp_copy""").show() 

+--------+
|count(1)|
+--------+
|      20|
+--------+



In [20]:
hc.sql("""select count(*) from office.employee""").show()

+--------+
|count(1)|
+--------+
|      10|
+--------+



In [21]:
hc.sql("""insert overwrite table office.emp_copy
        select * from office.employee""")

## this will overwrite the content of table emp_copy by employee

DataFrame[]

In [22]:
hc.sql("""select count(*) from office.emp_copy""").show()

+--------+
|count(1)|
+--------+
|      10|
+--------+



In [23]:
hc.sql("""select * from office.emp_copy where id = 1202""").show()

+----+--------+---+
|  id|    name|age|
+----+--------+---+
|1202| krishna| 28|
|1202|   krudi| 20|
|1202|  nrudvi| 29|
+----+--------+---+



In [24]:
hc.sql("""select * from office.emp_copy where id in (1202,1201)""").show()

+----+--------+---+
|  id|    name|age|
+----+--------+---+
|1201|  satish| 25|
|1202| krishna| 28|
|1202|   krudi| 20|
|1202|  nrudvi| 29|
|1201|  mrudvi| 35|
+----+--------+---+



In [25]:
hc.sql("""select * from office.employee where name rlike 's' """).show()

+----+--------+---+
|  id|    name|age|
+----+--------+---+
|1201|  satish| 25|
|1202| krishna| 28|
+----+--------+---+



In [26]:
hc.sql("""select *, 
        CASE WHEN age >= 20 and age < 24 THEN 'child'
            WHEN age >=24 and age < 28 THEN 'teenage'
            WHEN age >= 28 and age < 32 THEN 'adult'
            WHEN age >= 32 and age<36 then 'old'
            ELSE 'invalid'
        END AS age_category
        from office.employee""").show()

+----+--------+---+------------+
|  id|    name|age|age_category|
+----+--------+---+------------+
|1201|  satish| 25|     teenage|
|1202| krishna| 28|       adult|
|1203|   amith| 39|     invalid|
|1204|   javed| 23|       child|
|1205|  prudvi| 33|         old|
|1206|  drudvi| 26|     teenage|
|1202|   krudi| 20|       child|
|1203|  drudvi| 27|     teenage|
|1202|  nrudvi| 29|       adult|
|1201|  mrudvi| 35|         old|
+----+--------+---+------------+



In [27]:
hc.sql("""select distinct id from office.employee""").show()

+----+
|  id|
+----+
|1201|
|1206|
|1205|
|1203|
|1202|
|1204|
+----+



In [28]:
hc.sql("""select id, avg(age) from office.employee group by id """).show()

+----+------------------------+
|  id|avg(CAST(age AS DOUBLE))|
+----+------------------------+
|1201|                    30.0|
|1206|                    26.0|
|1205|                    33.0|
|1203|                    33.0|
|1202|      25.666666666666668|
|1204|                    23.0|
+----+------------------------+



In [29]:
hc.sql("""select id, avg(age) from office.employee group by id having avg(age) > 26""").show()

+----+------------------------+
|  id|avg(CAST(age AS DOUBLE))|
+----+------------------------+
|1201|                    30.0|
|1205|                    33.0|
|1203|                    33.0|
+----+------------------------+



In [30]:
## creates a file in the required location for a temporary query, for the future usage
hc.sql("""insert overwrite local directory 'testing_data' 
        row format delimited fields terminated by ',' 
        select id, avg(age) from office.employee group by id having avg(age)>26""")

DataFrame[]

In [31]:
hc.sql("""describe formatted office.emp_copy""").show(100,False)

+----------------------------+-------------------------------------------------------------+-------+
|col_name                    |data_type                                                    |comment|
+----------------------------+-------------------------------------------------------------+-------+
|id                          |int                                                          |null   |
|name                        |string                                                       |null   |
|age                         |string                                                       |null   |
|                            |                                                             |       |
|# Detailed Table Information|                                                             |       |
|Database                    |office                                                       |       |
|Table                       |emp_copy                                                     

In [32]:
hc.sql("""drop table office.emp_copy""")

DataFrame[]

In [33]:
%%sh
ls spark-warehouse/office.db/

employee


In [34]:
hc.sql("""CREATE EXTERNAL TABLE IF NOT EXISTS OFFICE.emp_copy(
        id INT, name STRING, age STRING
        ) 
        ROW FORMAT DELIMITED FIELDS TERMINATED BY ',' LINES TERMINATED BY '\n'
        location 'emp_copy'""")

DataFrame[]

In [35]:
%%sh
ls spark-warehouse/office.db/

emp_copy
employee


In [36]:
hc.sql("""describe formatted office.emp_copy""").show(100,False)

+----------------------------+-------------------------------------------------------------+-------+
|col_name                    |data_type                                                    |comment|
+----------------------------+-------------------------------------------------------------+-------+
|id                          |int                                                          |null   |
|name                        |string                                                       |null   |
|age                         |string                                                       |null   |
|                            |                                                             |       |
|# Detailed Table Information|                                                             |       |
|Database                    |office                                                       |       |
|Table                       |emp_copy                                                     

In [37]:
hc.sql("""select  * from office.emp_copy""").show()

+---+----+---+
| id|name|age|
+---+----+---+
+---+----+---+



In [38]:
hc.sql("""load data local inpath 'spark-warehouse/office.db/employee/employee.txt'
         into table office.emp_copy""")

DataFrame[]

In [39]:
hc.sql("""select * from office.emp_copy""").show()

+----+--------+---+
|  id|    name|age|
+----+--------+---+
|1201|  satish| 25|
|1202| krishna| 28|
|1203|   amith| 39|
|1204|   javed| 23|
|1205|  prudvi| 33|
|1206|  drudvi| 26|
|1202|   krudi| 20|
|1203|  drudvi| 27|
|1202|  nrudvi| 29|
|1201|  mrudvi| 35|
+----+--------+---+



In [40]:
%%sh
ls spark-warehouse/office.db/emp_copy/

employee.txt


In [41]:
## now if we drop the external table emp_copy, lets see the dataset if it gets dropped as well

hc.sql("""drop table office.emp_copy""")

DataFrame[]

In [42]:
%%sh
ls spark-warehouse/office.db/emp_copy/

employee.txt


In [43]:
## you can see the dataset doesnt get deleted, though the table and its structure gets dropped

hc.sql("""select * from office.emp_copy""").show()

AnalysisException: Table or view not found: office.emp_copy; line 1 pos 14;
'Project [*]
+- 'UnresolvedRelation [office, emp_copy], [], false


In [45]:
hc.sql("""create external table if not exists office.emp_copy_2 (id int, age string, name string)
        row format delimited fields terminated by ','
        location '../testing_external_table/emp_copy_2'""")

DataFrame[]

In [46]:
## now i have manually moved files to the external_table location, now it works

hc.sql("""select * from office.emp_copy_2""").show()

+----+--------+----+
|  id|     age|name|
+----+--------+----+
|1201|  satish|  25|
|1202| krishna|  28|
|1203|   amith|  39|
|1204|   javed|  23|
|1205|  prudvi|  33|
|1206|  drudvi|  26|
|1202|   krudi|  20|
|1203|  drudvi|  27|
|1202|  nrudvi|  29|
|1201|  mrudvi|  35|
+----+--------+----+



In [47]:
hc.sql("""describe formatted office.emp_copy_2""").show(100,False)

+----------------------------+----------------------------------------------------------------------------+-------+
|col_name                    |data_type                                                                   |comment|
+----------------------------+----------------------------------------------------------------------------+-------+
|id                          |int                                                                         |null   |
|age                         |string                                                                      |null   |
|name                        |string                                                                      |null   |
|                            |                                                                            |       |
|# Detailed Table Information|                                                                            |       |
|Database                    |office                                    

In [48]:
hc.sql("""insert overwrite local directory 'testing_data/' row format delimited fields terminated by ','
        select * from office.employee sort by id desc""")

DataFrame[]

In [49]:
hc.sql("set mapreduce.job.reduce = 3")

DataFrame[key: string, value: string]

In [50]:
hc.sql("""insert overwrite local directory 'testing_data/test' row format delimited fields terminated by ','
        select * from office.employee sort by id desc""")

DataFrame[]

In [51]:
hc.sql("""insert overwrite local directory 'testing_data/test_distribute' row format delimited 
        fields terminated by ',' 
        select * from office.employee 
        distribute by id 
        sort by id desc""")

DataFrame[]

In [53]:
# this is similar to distribute by + sort by, the only need is we need to use same set of columns in 
# distrubute by and sort by in order to have same results 
hc.sql("""insert overwrite local directory 'testing_data/test_distribute/' row format delimited 
        fields terminated by ','
        select * from office.employee
        cluster by id""")

DataFrame[]

### partitions

In [55]:
hc.sql("""create table if not exists office.partition_emp (name STRING, age STRING)
        partitioned by (id INT)
        row format delimited fields terminated by ','
        """)

21/11/07 13:42:56 WARN HiveMetaStore: Location: file:/home/shivam/notebook/spark-warehouse/office.db/partition_emp specified for non-external table:partition_emp


DataFrame[]

In [56]:
hc.sql("""describe formatted office.partition_emp""").show(100,False)

+----------------------------+------------------------------------------------------------------+-------+
|col_name                    |data_type                                                         |comment|
+----------------------------+------------------------------------------------------------------+-------+
|name                        |string                                                            |null   |
|age                         |string                                                            |null   |
|id                          |int                                                               |null   |
|# Partition Information     |                                                                  |       |
|# col_name                  |data_type                                                         |comment|
|id                          |int                                                               |null   |
|                            |                

In [58]:
# inserting specific partition from target table
hc.sql("""insert overwrite table office.partition_emp
        partition (id = 1201)
        select name,age from office.employee
        where id = 1201""")

DataFrame[]

In [59]:
hc.sql("""select * from office.partition_emp""").show()

+-------+---+----+
|   name|age|  id|
+-------+---+----+
| satish| 25|1201|
| mrudvi| 35|1201|
+-------+---+----+



In [60]:
hc.sql("""insert overwrite table office.partition_emp
        partition (id = 1202)
        select name,age from office.employee
        where id=1202""")

DataFrame[]

In [61]:
hc.sql("select * from office.partition_emp").show()

+--------+---+----+
|    name|age|  id|
+--------+---+----+
|  satish| 25|1201|
|  mrudvi| 35|1201|
| krishna| 28|1202|
|   krudi| 20|1202|
|  nrudvi| 29|1202|
+--------+---+----+



In [63]:
hc.sql("""show partitions office.partition_emp""").show()

+---------+
|partition|
+---------+
|  id=1201|
|  id=1202|
+---------+



In [80]:
hc.sql("""describe formatted office.partition_emp""").show(100,False)

+----------------------------+------------------------------------------------------------------+-------+
|col_name                    |data_type                                                         |comment|
+----------------------------+------------------------------------------------------------------+-------+
|name                        |string                                                            |null   |
|age                         |string                                                            |null   |
|id                          |int                                                               |null   |
|# Partition Information     |                                                                  |       |
|# col_name                  |data_type                                                         |comment|
|id                          |int                                                               |null   |
|                            |                

In [66]:
%%sh
ls /home/shivam/notebook/spark-warehouse/office.db/partition_emp

id=1201
id=1202


In [81]:
hc.sql("""insert overwrite local directory '/home/shivam/notebook/spark-warehouse/office.db/partition_emp/id=1203'
        row format delimited fields terminated by ','
        select name,age from office.employee where id=1203""")

DataFrame[]

In [82]:
hc.sql("""select * from office.partition_emp""").show()

+--------+---+----+
|    name|age|  id|
+--------+---+----+
|  satish| 25|1201|
|  mrudvi| 35|1201|
| krishna| 28|1202|
|   krudi| 20|1202|
|  nrudvi| 29|1202|
+--------+---+----+



In [83]:
%%sh
ls /home/shivam/notebook/spark-warehouse/office.db/partition_emp

id=1201
id=1202
id=1203


In [84]:
## this will just create a directory with different name ,,, thats it, the partition is not added yet
hc.sql("""select * from office.partition_emp where id=1203""").show()

+----+---+---+
|name|age| id|
+----+---+---+
+----+---+---+



In [85]:
## we need to add partition to the table using alter cmd
hc.sql("""alter table office.partition_emp add if not exists 
            partition (id=1203) location '/home/shivam/notebook/spark-warehouse/office.db/partition_emp/id=1203'
    """)

21/11/07 14:16:41 WARN log: Updating partition stats fast for: partition_emp
21/11/07 14:16:41 WARN log: Updated size to 23


DataFrame[]

In [87]:
## now the partition is added and we can now show the partitions in the table
hc.sql("""select * from office.partition_emp""").show()

+--------+---+----+
|    name|age|  id|
+--------+---+----+
|  satish| 25|1201|
|  mrudvi| 35|1201|
| krishna| 28|1202|
|   krudi| 20|1202|
|  nrudvi| 29|1202|
|   amith| 39|1203|
|  drudvi| 27|1203|
+--------+---+----+



In [89]:
hc.sql("""show partitions office.partition_emp""").show()

+---------+
|partition|
+---------+
|  id=1201|
|  id=1202|
|  id=1203|
+---------+



In [90]:
hc.sql("""alter table office.partition_emp drop if exists partition (id=1203)""")

DataFrame[]

In [91]:
hc.sql("""show partitions office.partition_emp""").show()

+---------+
|partition|
+---------+
|  id=1201|
|  id=1202|
+---------+



In [94]:
# the first config is deprecated, when using hive in spark , prefix spark in the config
# hc.sql("""set hive.exec.dynamic.partition=true""")
hc.sql("""set spark.hadoop.hive.exec.dynamic.partition=true""")

DataFrame[key: string, value: string]

In [98]:
hc.sql("""set spark.hadoop.hive.exec.dynamic.partition.mode=nonstrict""")

DataFrame[key: string, value: string]

In [102]:
hc.sql("""insert overwrite table office.partition_emp
        partition (id)
        select name,age,id from office.employee""")

21/11/07 14:36:16 WARN HiveConf: HiveConf of name hive.internal.ss.authz.settings.applied.marker does not exist
21/11/07 14:36:16 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
21/11/07 14:36:16 WARN HiveConf: HiveConf of name hive.stats.retries.wait does not exist
21/11/07 14:36:16 WARN HiveConf: HiveConf of name hive.internal.ss.authz.settings.applied.marker does not exist
21/11/07 14:36:16 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
21/11/07 14:36:16 WARN HiveConf: HiveConf of name hive.stats.retries.wait does not exist
21/11/07 14:36:16 WARN HiveConf: HiveConf of name hive.internal.ss.authz.settings.applied.marker does not exist
21/11/07 14:36:16 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
21/11/07 14:36:16 WARN HiveConf: HiveConf of name hive.stats.retries.wait does not exist
21/11/07 14:36:16 WARN HiveConf: HiveConf of name hive.internal.ss.authz.settings.applied.marker does not exist
21/11/07 14:36:16 

DataFrame[]

## configure the dynamic partitions
hc.sql("""set spark.hadoop.hive.exec.max.dynamic.partitions=1000""")
hc.sql("""set spark.hadoop.hive.exec.max.dynamic.partitions.pernode=500""")

In [104]:
hc.sql("""select * from office.partition_emp where age=25""").show()

+-------+---+----+
|   name|age|  id|
+-------+---+----+
| satish| 25|1201|
+-------+---+----+



In [105]:
hc.sql("""select * from office.employee""").show()

+----+--------+---+
|  id|    name|age|
+----+--------+---+
|1201|  satish| 25|
|1202| krishna| 28|
|1203|   amith| 39|
|1204|   javed| 23|
|1205|  prudvi| 33|
|1206|  drudvi| 26|
|1202|   krudi| 20|
|1203|  drudvi| 27|
|1202|  nrudvi| 29|
|1201|  mrudvi| 35|
+----+--------+---+



In [110]:
hc.sql("""select id,name,
                avg(age) over(PARTITION BY id ) 
                as running_avg
        from office.employee""").show()

## first the dataset is partitioned or grouped by age , this is known as window
## and next analytical function gets executed for  each record
## the average is still calculated for each record in the partition or window but it is executed for each record


+----+--------+------------------+
|  id|    name|       running_avg|
+----+--------+------------------+
|1201|  satish|              30.0|
|1201|  mrudvi|              30.0|
|1206|  drudvi|              26.0|
|1205|  prudvi|              33.0|
|1203|   amith|              33.0|
|1203|  drudvi|              33.0|
|1202| krishna|25.666666666666668|
|1202|   krudi|25.666666666666668|
|1202|  nrudvi|25.666666666666668|
|1204|   javed|              23.0|
+----+--------+------------------+



In [111]:
hc.sql("""select id,name,
                avg(age) over(PARTITION BY id rows between unbounded preceding and current row) 
                as running_avg
        from office.employee""").show()

## first the dataset is partitioned or grouped by age , this is known as window
## and then frames are generated that would include the current row and the rows behind it
## next analytical function gets executed for  each frame generated

+----+--------+------------------+
|  id|    name|       running_avg|
+----+--------+------------------+
|1201|  satish|              25.0|
|1201|  mrudvi|              30.0|
|1206|  drudvi|              26.0|
|1205|  prudvi|              33.0|
|1203|   amith|              39.0|
|1203|  drudvi|              33.0|
|1202| krishna|              28.0|
|1202|   krudi|              24.0|
|1202|  nrudvi|25.666666666666668|
|1204|   javed|              23.0|
+----+--------+------------------+



In [ ]:
hc.sql("""select ymd, exchange, symbol, volume, price_close,
            avg(price_close) over(PARTITION BY symbol ORDER BY ymd
                                ROWS BETWEEN 9 PRECEDING AND CURRENT ROW) AS 10_DAY_MOVING_AVG
        from stocks
        where symbol in ('abc','def')""")

In [117]:
hc.sql("""create table if not exists office.emp_old_age as
        select id, name, age from (
        select * ,
            row_number() over (partition by id order by age desc) as row_number
        from office.employee
        )
        
        where row_number=1""")

21/11/08 00:12:32 WARN ResolveSessionCatalog: A Hive serde table will be created as there is no table provider specified. You can set spark.sql.legacy.createHiveTableByDefault to false so that native data source table will be created instead.
21/11/08 00:12:32 WARN HiveMetaStore: Location: file:/home/shivam/notebook/spark-warehouse/office.db/emp_old_age specified for non-external table:emp_old_age


DataFrame[]

In [118]:
hc.sql("""select * from office.emp_old_age""").show()

+----+-------+---+
|  id|   name|age|
+----+-------+---+
|1201| mrudvi| 35|
|1203|  amith| 39|
|1202| nrudvi| 29|
|1204|  javed| 23|
|1206| drudvi| 26|
|1205| prudvi| 33|
+----+-------+---+



In [120]:
## lead checks for the value in the attribute in the next record of  the window,
## lag checks for the value in the attribute in the previous record of the window.
hc.sql("""select *,
        row_number() over(partition by id order by age) as rank,
        lag(age,1) over(partition by id order by age) as previous_age,
        lead(age,1) over(partition by id order by age) as after_age
    from office.employee""").show()

+----+--------+---+----+------------+---------+
|  id|    name|age|rank|previous_age|after_age|
+----+--------+---+----+------------+---------+
|1201|  satish| 25|   1|        null|       35|
|1201|  mrudvi| 35|   2|          25|     null|
|1206|  drudvi| 26|   1|        null|     null|
|1205|  prudvi| 33|   1|        null|     null|
|1203|  drudvi| 27|   1|        null|       39|
|1203|   amith| 39|   2|          27|     null|
|1202|   krudi| 20|   1|        null|       28|
|1202| krishna| 28|   2|          20|       29|
|1202|  nrudvi| 29|   3|          28|     null|
|1204|   javed| 23|   1|        null|     null|
+----+--------+---+----+------------+---------+

